# Game Dataset Cleaning
This notebook prepares the RAWG-based video game dataset for analysis by:
- Removing irrelevant or duplicate rows and columns
- Properly parsing nested data (genres, platforms, stores)
- Creating engineered features (platform type, rating gap, etc.)
- Saving a cleaned version for further analysis

### Load Data

In [1]:

import pandas as pd
import numpy as np
import sys
import ast

# For plotting functions
sys.path.append("..")

games = pd.read_csv("../data/games_data.csv")

games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  10000 non-null  int64  
 1   name                10000 non-null  object 
 2   released            9842 non-null   object 
 3   avg_user_rating     10000 non-null  float64
 4   user_ratings_count  10000 non-null  int64  
 5   reviews_count       10000 non-null  int64  
 6   added               10000 non-null  int64  
 7   metacritic_rating   4250 non-null   float64
 8   avg_playtime        10000 non-null  int64  
 9   genres              10000 non-null  object 
 10  platforms           10000 non-null  object 
 11  stores              10000 non-null  object 
 12  tags                10000 non-null  object 
 13  esrb_rating         4281 non-null   object 
dtypes: float64(2), int64(5), object(7)
memory usage: 1.1+ MB


### Check and Clean Basic Missing Values

In [2]:
# Check for missing values
print(games.isna().sum())

# Drop duplicates
games.drop_duplicates(inplace = True)

# Drop rows with non release date
games = games.dropna(subset= ['released']).copy()

id                       0
name                     0
released               158
avg_user_rating          0
user_ratings_count       0
reviews_count            0
added                    0
metacritic_rating     5750
avg_playtime             0
genres                   0
platforms                0
stores                   0
tags                     0
esrb_rating           5719
dtype: int64


### Feature Engineering - Release Date Components

In [3]:
games['released'] = pd.to_datetime(games['released'], format = '%Y-%m-%d')

# Extract date components
games['release_year'] = games['released'].dt.year
games['release_month'] = games['released'].dt.month
games['release_quarter'] = games['released'].dt.quarter

### Feature Engineering - Critic and User Rating Gap (Extra/Supplementary)

In [4]:
games['critic_rating'] = games['metacritic_rating'] / 20
games['rating_gap'] = games['avg_user_rating'] - games['critic_rating']


### Genre Processing

In [5]:
games['genres'] = games['genres'].apply(ast.literal_eval)
games['genre_count'] = games['genres'].apply(len)


# Binarize Genres
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
encoded_genres = pd.DataFrame(mlb.fit_transform(games['genres']), columns=[f"is_{g.lower().replace(' ', '_')}" for g in mlb.classes_],
                             index=games.index)

games = pd.concat([games, encoded_genres], axis=1)
games.drop(columns=['genres'], inplace=True)

### Platform Features

In [6]:
def platform_type(platforms):
    pc = any('PC' in p for p in platforms)
    console = any(any(c in p for c in ['PlayStation', 'Xbox', 'Nintendo']) for p in platforms)

    if pc and not console:
        return 'pc_only'
    elif console and not pc:
        return 'console_only'
    elif pc and console:
        return 'multi_platform'
    else:
        return 'other' 
    
    
games['platforms'] = games['platforms'].apply(ast.literal_eval)
games['platform_type'] = games['platforms'].apply(platform_type)
games['platform_count'] =  games['platforms'].apply(len)



### Store Features

In [7]:
games['stores'] = games['stores'].apply(ast.literal_eval)
games['store_count'] = games['stores'].apply(len)


### Drop unneeded columns and Metadata errors from API

In [8]:
games.drop(columns = ['released', 'metacritic_rating','tags', 'stores', 'platforms'], inplace= True)

# Metadata error
games = games[games['genre_count'] != 0]
games = games[games['store_count']!= 0]
games = games[games['platform_count'] != 0]

# Games only before 2025 to remove skew
games= games[games['release_year']< 2025]

# Only keep games with at least 10 user ratings
games = games[games['user_ratings_count'] >= 10]

games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8122 entries, 0 to 9999
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        8122 non-null   int64  
 1   name                      8122 non-null   object 
 2   avg_user_rating           8122 non-null   float64
 3   user_ratings_count        8122 non-null   int64  
 4   reviews_count             8122 non-null   int64  
 5   added                     8122 non-null   int64  
 6   avg_playtime              8122 non-null   int64  
 7   esrb_rating               3904 non-null   object 
 8   release_year              8122 non-null   int32  
 9   release_month             8122 non-null   int32  
 10  release_quarter           8122 non-null   int32  
 11  critic_rating             3956 non-null   float64
 12  rating_gap                3956 non-null   float64
 13  genre_count               8122 non-null   int64  
 14  is_action    

### Inspect and Save Cleaned Data

In [9]:
games.head(20)

,id,name,avg_user_rating,user_ratings_count,reviews_count,added,avg_playtime,esrb_rating,release_year,release_month,...,is_puzzle,is_rpg,is_racing,is_shooter,is_simulation,is_sports,is_strategy,platform_type,platform_count,store_count
0,3498,Grand Theft Auto V,4.47,7160,7275,22138,74,Mature,2013,9,...,0,0,0,0,0,0,0,multi_platform,7,5
1,3328,The Witcher 3: Wild Hunt,4.64,6964,7077,21799,43,Mature,2015,5,...,0,1,0,0,0,0,0,multi_platform,7,5
2,4200,Portal 2,4.59,5942,5998,20628,11,Everyone 10+,2011,4,...,1,0,0,1,0,0,0,multi_platform,6,4
3,4291,Counter-Strike: Global Offensive,3.57,3586,3624,18286,64,Mature,2012,8,...,0,0,0,1,0,0,0,multi_platform,4,3
4,5286,Tomb Raider (2013),4.06,4022,4054,17635,10,Mature,2013,3,...,0,0,0,0,0,0,0,multi_platform,6,6
5,13536,Portal,4.49,4898,4942,17587,4,Teen,2007,10,...,1,0,0,0,0,0,0,multi_platform,7,2
6,12020,Left 4 Dead 2,4.09,3356,3381,17401,9,Mature,2009,11,...,0,0,0,1,0,0,0,multi_platform,4,2
7,5679,The Elder Scrolls V: Skyrim,4.42,4803,4856,16596,46,Mature,2011,11,...,0,1,0,0,0,0,0,multi_platform,8,4
8,28,Red Dead Redemption 2,4.59,5296,5420,16462,21,Mature,2018,10,...,0,0,0,0,0,0,0,multi_platform,3,4
9,4062,BioShock Infinite,4.38,4122,4164,15967,12,Mature,2013,3,...,0,0,0,1,0,0,0,multi_platform,7,7


In [10]:
games.to_csv("..\data\games_data_cleaned.csv", index = False)
